In [149]:
import pandas as pd
import numpy as np
import os, sys, inspect, time
import folium
import subprocess
from folium.plugins import HeatMap
from sklearn import preprocessing
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


# add parent dir to system dir
currdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootdir = os.path.dirname(currdir)
sys.path.insert(0, rootdir)

from src.utils import get_temporal_data, get_spatial_data

In [150]:
# read main real state df
ffname_realestate = os.path.join(rootdir, "data", "int", "realestate_demographics.csv")
realestate_df = pd.read_csv(ffname_realestate)
realestate_df.columns

Index(['Zip Code', 'Year', 'Month', 'Home Value Index', 'Median Listing Price',
       'Median Listing Price Per sqf', 'Sales', 'Median Income (dollars)',
       'Number of Households', 'School Rating (Parent)', 'School Rating(GS)',
       'Mortgage Rate_15', 'Mortgage Rate_30', 'Total_NewConstructions',
       'Total_RemodelsRepairs', 'TotalCost_NewConstructions',
       'TotalCost_RemodelRepairs', 'Bank Count', 'Liquor Store Count',
       'Wal-Mart Count', 'Starbucks Count', 'Target Count',
       'Inventory Measure', 'Median Price Cut', 'Median Price of Reduction',
       'Median Rental Price', 'Latitude', 'Longitude'],
      dtype='object')

In [165]:
# set indicator to plot
column_to_choroplethmap = "Home Value Index"
# column_to_choroplethmap = "Median Listing Price"
# column_to_choroplethmap = "Median Listing Price Per sqf"
# column_to_choroplethmap = "Median Income (dollars)"
# column_to_choroplethmap = "Number of Households"
# column_to_choroplethmap = "School Rating (Parent)"
# column_to_choroplethmap = "School Rating(GS)"
# column_to_choroplethmap = "Total_NewConstructions"
# column_to_choroplethmap = "Total_RemodelsRepairs"
# column_to_choroplethmap = "TotalCost_NewConstructions"
# column_to_choroplethmap = "Bank Count"
# column_to_choroplethmap = "Liquor Store Count"
# column_to_choroplethmap = "Wal-Mart Count"
# column_to_choroplethmap = "Starbucks Count"
# column_to_choroplethmap = "Target Count"
# column_to_choroplethmap = "Inventory Measure"

year = 2016
month = 12

df = get_spatial_data(year, realestate_df)
df = df.loc[df["Month"] == month, :]
df = df[["Zip Code", column_to_choroplethmap]]
df.reset_index(inplace=True)
df.drop(["index"], inplace=True, axis=1)
df["Zip Code"] = df["Zip Code"].astype(str)
df.dropna(inplace=True)
df.head()

,Zip Code,Home Value Index
0,78701,366600.0
1,78702,375200.0
2,78703,741800.0
3,78704,472100.0
4,78705,249600.0


In [166]:
# set choropleth map params
location = [30.3100, -97.8]
zoom_start = 11
name = column_to_choroplethmap
radius = 30

# rescale to [0, 1]
# max_val = df[column_to_choroplethmap].max()
# min_val = df[column_to_choroplethmap].min()
# df[column_to_choroplethmap] = (df[column_to_choroplethmap] - min_val) / (max_val - min_val)
# creat threshold range
max_val = df[column_to_choroplethmap].max()
min_val = df[column_to_choroplethmap].min()
threshold_scale = np.linspace(min_val, max_val, 6).tolist()
# set fill color
fill_color = "BuPu"

In [167]:
# create map
m = folium.Map(location, zoom_start=zoom_start)
ffname = os.path.join(rootdir, "data", "ext", "austin-zipcode-boundaries.geojson")
# create choropleth map
m.choropleth(
    geo_data=ffname,
    data=df,
    columns=["Zip Code", column_to_choroplethmap],
    key_on="feature.properties.zipcode",
    fill_color=fill_color)
m

In [168]:
# save built map as html
fname = f"choroplethmap-" + name.replace(" ", "-").lower().replace(")", "").replace("(", "") + f"-{year}-{month}"
m.save(os.path.join(rootdir, 'reports', 'figures', fname + ".html"))

In [169]:
# save to png file (by cutycapt)
# fname = f"choroplethmap-" + name.replace(" ", "-").lower().replace(")", "").replace("(", "") + f"-{year}-{month}"
# url = f"file://{rootdir}/reports/figures/{fname}.html"
# ffname = os.path.join(rootdir, "reports", "figures", fname + ".png" )
# subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(ffname)])

In [170]:
# save to png file (by selenium)
fname = f"choroplethmap-" + name.replace(" ", "-").lower().replace(")", "").replace("(", "") + f"-{year}-{month}"
url = f"file://{rootdir}/reports/figures/{fname}" + ".html"
ffname = os.path.join(rootdir, "reports", "figures", fname + ".png" )
browser = webdriver.Chrome("/usr/local/bin/chromedriver")
browser.get(url)

In [ ]:
time.sleep(1)  #Give the map tiles some time to load
browser.save_screenshot(ffname)
browser.quit()